In [39]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from peft import LoraConfig, TaskType, get_peft_model
import plotly.graph_objects as go
from datasets import Dataset
import pandas as pd
import numpy as np
import evaluate
import evaluate
import json
import re

In [ ]:
def process_jsonl(input_file, tokenizer, sentiment_mapping):
    """
    Processes a JSONL file and returns a list of dictionaries with tokens and labels.
    
    Args:
        input_file (str): Path to the input JSONL file.
        tokenizer (AutoTokenizer): Hugging Face tokenizer.
        sentiment_mapping (dict): Mapping from sentiment strings to standardized labels.
        
    Returns:
        list: A list of dictionaries with 'tokens' and 'labels'.
    """
    def clean_word(word):
        """Remove unnecessary spaces from tokens."""
        # Strip only spaces, not punctuation
        return word.strip()

    def split_text_into_tokens(text):
        """
        Splits text into tokens, treating punctuation as separate tokens.
        Example: "Hello, world!" -> ["Hello", ",", "world", "!"]
        """
        return re.findall(r'\w+|[^\w\s]', text, re.UNICODE)

    # Open and read the input JSONL file
    with open(input_file, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]

    processed_data = []

    # Iterate through each review in the data
    for item in data:
        text = item['text']
        labels = item.get('label', [])  # Use .get() to handle missing 'label' fields

        # Tokenize text, splitting punctuation into separate tokens
        tokens = split_text_into_tokens(text)
        token_offsets = []
        current_pos = 0

        # Find character positions for each token
        for token in tokens:
            start = text.find(token, current_pos)
            end = start + len(token)
            token_offsets.append((start, end))
            current_pos = end

        # Initialize labels for each token as "O"
        token_labels = ["O"] * len(tokens)

        # Assign labels based on sentiment spans
        for start, end, sentiment in labels:
            # Standardize sentiment label
            sentiment_standard = sentiment_mapping.get(sentiment, "O")
            if sentiment_standard == "O":
                continue  # Skip if sentiment is not recognized

            for i, (token_start, token_end) in enumerate(token_offsets):
                if token_start >= start and token_end <= end:
                    if token_start == start:
                        token_labels[i] = f"B-{sentiment_standard}"
                    else:
                        token_labels[i] = f"I-{sentiment_standard}"

        # Clean tokens and remove any that are empty after cleaning
        cleaned_tokens = [clean_word(token) for token in tokens]
        cleaned_tokens, token_labels = zip(*[
            (token, label) for token, label in zip(cleaned_tokens, token_labels) if token
        ])

        # Append the processed entry
        processed_data.append({
            "tokens": list(cleaned_tokens),
            "labels": list(token_labels)
        })

    return processed_data


# Example usage:
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased", use_fast=True)
sentiment_mapping = {
    'Negative': 'Negative',
    'Neutral': 'Neutral',
    'Positive': 'Positive'
}
processed_data = process_jsonl("patryk.jsonl", tokenizer, sentiment_mapping)

In [41]:
# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(pd.DataFrame(processed_data))
print("Dataset example:")
print(dataset[0])

Dataset example:
{'tokens': ['Lakier', 'roweru', 'bardzo', 'kiepskiej', 'jakości', 'robią', 'się', 'odpryski', 'nie', 'wiadomo', 'od', 'czego', 'rower', 'ładny', 'wygodny', 'ale', 'po', '3', 'miesiącach', 'użytkowania', 'widoczne', 'odpryski', 'lakieru', 'czego', 'za', 'taką', 'cenę', 'nie', 'powinno', 'być', 'Oczywiście', 'producent', 'twierdzi', 'że', 'są', 'to', 'wady', 'mechaniczne', 'dziecko', 'ma', 'w', 'lepszym', 'stanie', 'lakier', 'na', 'rowerze', 'ale', 'nie', 'z', 'tej', 'firmy', 'ODRADZAM', 'ZAKUP', 'Z', 'TEGO', 'POWODU', 'SZKODA', 'TYLE', 'KASY', 'I', 'NERWÓW', 'chyba', 'ze', 'rower', 'będzie', 'stał', 'nieużywany', 'za', 'szybą', 'Na', 'zakończenie', 'powiem', 'tak', 'porównując', 'lakier', 'zwykły', 'do', 'paznokci', 'a', 'hybrydę', 'wiadomo', 'w', 'tańszym', 'zwykłym', 'lakierze', 'robią', 'się', 'odpryski', 'a', 'lepszym', 'nie'], 'labels': ['O', 'O', 'O', 'B-Negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Positive', 'B-Positive', 'O', 'O', 'O', 'O', 'O', 'B

In [42]:
# Define label list including "O" for tokens outside any entity
label_list = ["O", "B-Negative", "I-Negative", "B-Positive", "I-Positive", "B-Neutral", "I-Neutral"]

# Create mappings from label to ID and ID to label
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

print("Label to ID Mapping:")
print(label_to_id)

print("\nID to Label Mapping:")
print(id_to_label)

Label to ID Mapping:
{'O': 0, 'B-Negative': 1, 'I-Negative': 2, 'B-Positive': 3, 'I-Positive': 4, 'B-Neutral': 5, 'I-Neutral': 6}

ID to Label Mapping:
{0: 'O', 1: 'B-Negative', 2: 'I-Negative', 3: 'B-Positive', 4: 'I-Positive', 5: 'B-Neutral', 6: 'I-Neutral'}


In [43]:
from transformers import DataCollatorForTokenClassification

def tokenize_and_align_labels(examples):
    """
    Tokenizes the input texts and aligns the labels with the tokens.
    
    Args:
        examples (dict): Dictionary containing 'tokens' and 'labels'.
        
    Returns:
        dict: Tokenized inputs with aligned labels.
    """
    tokenized_inputs = tokenizer(
        examples['tokens'],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_offsets_mapping=True
    )
    
    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to words
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens
            elif word_idx != previous_word_idx:
                # Beginning of a word
                label_ids.append(label_to_id.get(label[word_idx], 0))
            else:
                # Inside a word
                if label[word_idx].startswith("B-"):
                    label_ids.append(label_to_id.get(label[word_idx].replace("B-", "I-"), 0))
                else:
                    label_ids.append(label_to_id.get(label[word_idx], 0))
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the tokenization and alignment
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=['tokens', 'labels']
)

print("Tokenized Dataset Example:")
print(tokenized_datasets[0])

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenized Dataset Example:
{'labels': [-100, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 4, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, -100, -100, -100, -100], 'input_ids': [0, 4071, 13281, 40876, 2450, 17696, 2257, 8842, 10362, 2022, 2021, 7046, 2113, 1997, 4610, 2173, 2784, 20290, 31965, 12858, 2067, 2199, 2184, 1034, 10947, 18496, 15568, 2021, 7046, 2113, 38534, 4236, 2784, 2163, 5117, 6598, 1997, 5606, 2458, 5581, 16309, 4869, 2040, 2264, 2063, 19225, 46568, 6067, 2185, 1019, 20465, 3110, 2050, 13281, 1998, 23922, 2199, 1997, 1046, 2320, 3780, 9993, 3864, 5327, 14609, 7813, 6901, 1087, 1051, 45468, 52, 36917, 1041, 30729, 11452, 8203, 16523, 6132, 27649, 1056, 41785, 59, 5633, 4242, 2343, 20290, 2282, 3640, 

In [44]:
# Split the dataset into training and evaluation sets (e.g., 80% train, 20% test)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2, seed=42)

# Access the 'train' and 'test' splits
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['test']

print(f"\nNumber of training samples: {len(train_dataset)}")
print(f"Number of evaluation samples: {len(eval_dataset)}")


Number of training samples: 240
Number of evaluation samples: 60


In [45]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [46]:
# Initialize the model
foundation_model = AutoModelForTokenClassification.from_pretrained(
    "allegro/herbert-base-cased",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)
foundation_model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [47]:
for name, module in foundation_model.named_modules():
    print(name)


bert
bert.embeddings
bert.embeddings.word_embeddings
bert.embeddings.position_embeddings
bert.embeddings.token_type_embeddings
bert.embeddings.LayerNorm
bert.embeddings.dropout
bert.encoder
bert.encoder.layer
bert.encoder.layer.0
bert.encoder.layer.0.attention
bert.encoder.layer.0.attention.self
bert.encoder.layer.0.attention.self.query
bert.encoder.layer.0.attention.self.key
bert.encoder.layer.0.attention.self.value
bert.encoder.layer.0.attention.self.dropout
bert.encoder.layer.0.attention.output
bert.encoder.layer.0.attention.output.dense
bert.encoder.layer.0.attention.output.LayerNorm
bert.encoder.layer.0.attention.output.dropout
bert.encoder.layer.0.intermediate
bert.encoder.layer.0.intermediate.dense
bert.encoder.layer.0.intermediate.intermediate_act_fn
bert.encoder.layer.0.output
bert.encoder.layer.0.output.dense
bert.encoder.layer.0.output.LayerNorm
bert.encoder.layer.0.output.dropout
bert.encoder.layer.1
bert.encoder.layer.1.attention
bert.encoder.layer.1.attention.self
bert.e

In [48]:

print(list(TaskType))

lora_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,          # Correct task type for token-level tasks
    r=64,                                  # Rank of LoRA; adjust as needed
    lora_alpha=32,                         # Scaling factor; adjust as needed
    lora_dropout=0.05,                     # Dropout probability
    # target_modules=["classifier"]           # Correct target module(s)
)

[<TaskType.SEQ_CLS: 'SEQ_CLS'>, <TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>, <TaskType.CAUSAL_LM: 'CAUSAL_LM'>, <TaskType.TOKEN_CLS: 'TOKEN_CLS'>, <TaskType.QUESTION_ANS: 'QUESTION_ANS'>, <TaskType.FEATURE_EXTRACTION: 'FEATURE_EXTRACTION'>]


In [49]:

peft_model = get_peft_model(foundation_model, lora_config)

peft_model.print_trainable_parameters()

trainable params: 2,364,679 || all params: 126,222,350 || trainable%: 1.8734


In [50]:
peft_model

PeftModelForTokenClassification(
  (base_model): LoraModel(
    (model): BertForTokenClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(50000, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Li

In [51]:


metric = evaluate.load("seqeval")

def compute_metrics(p):
    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id_to_label[label] for label in label_seq if label != -100]
        for label_seq in labels
    ]
    true_predictions = [
        [id_to_label[pred] for (pred, label) in zip(pred_seq, label_seq) if label != -100]
        for pred_seq, label_seq in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

/opt/miniconda3/envs/py310/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning:

`evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead



In [52]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate()
print("\nEvaluation Results:")
print(results)

  0%|          | 0/150 [00:00<?, ?it/s]

{'loss': 2.0447, 'grad_norm': 15.333028793334961, 'learning_rate': 1.866666666666667e-05, 'epoch': 0.67}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.971683144569397, 'eval_precision': 0.0074285714285714285, 'eval_recall': 0.09848484848484848, 'eval_f1': 0.01381509032943677, 'eval_accuracy': 0.0651085141903172, 'eval_runtime': 0.6261, 'eval_samples_per_second': 95.83, 'eval_steps_per_second': 6.389, 'epoch': 1.0}
{'loss': 1.9752, 'grad_norm': 16.2625675201416, 'learning_rate': 1.7333333333333336e-05, 'epoch': 1.33}
{'loss': 1.9026, 'grad_norm': 15.829793930053711, 'learning_rate': 1.6000000000000003e-05, 'epoch': 2.0}


  0%|          | 0/4 [00:00<?, ?it/s]

...

{'eval_loss': 1.4115933179855347, 'eval_precision': 0.007462686567164179, 'eval_recall': 0.007575757575757576, 'eval_f1': 0.007518796992481203, 'eval_accuracy': 0.8593489148580968, 'eval_runtime': 0.633, 'eval_samples_per_second': 94.79, 'eval_steps_per_second': 6.319, 'epoch': 9.0}
{'loss': 1.4197, 'grad_norm': 13.67158031463623, 'learning_rate': 1.3333333333333334e-06, 'epoch': 9.33}
{'loss': 1.4021, 'grad_norm': 13.221175193786621, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.4027934074401855, 'eval_precision': 0.007874015748031496, 'eval_recall': 0.007575757575757576, 'eval_f1': 0.007722007722007722, 'eval_accuracy': 0.8614357262103506, 'eval_runtime': 0.6053, 'eval_samples_per_second': 99.118, 'eval_steps_per_second': 6.608, 'epoch': 10.0}
{'train_runtime': 65.5028, 'train_samples_per_second': 36.64, 'train_steps_per_second': 2.29, 'train_loss': 1.6505492464701335, 'epoch': 10.0}


  0%|          | 0/4 [00:00<?, ?it/s]


Evaluation Results:
{'eval_loss': 1.6015957593917847, 'eval_precision': 0.011725293132328308, 'eval_recall': 0.05303030303030303, 'eval_f1': 0.019204389574759947, 'eval_accuracy': 0.6690317195325542, 'eval_runtime': 0.6202, 'eval_samples_per_second': 96.748, 'eval_steps_per_second': 6.45, 'epoch': 10.0}


In [53]:
results = trainer.evaluate()
print("\nEvaluation Results:")
print(results)

  0%|          | 0/4 [00:00<?, ?it/s]


Evaluation Results:
{'eval_loss': 1.6015957593917847, 'eval_precision': 0.011725293132328308, 'eval_recall': 0.05303030303030303, 'eval_f1': 0.019204389574759947, 'eval_accuracy': 0.6690317195325542, 'eval_runtime': 0.6415, 'eval_samples_per_second': 93.526, 'eval_steps_per_second': 6.235, 'epoch': 10.0}


In [54]:
from transformers import pipeline

nlp = pipeline(
    "token-classification",
    model=peft_model,
    tokenizer=tokenizer,
    aggregation_strategy="none"
)

inference_results = []

example_texts = [
    "Nie jestem zadowolony z zakupu. Słuchawki są niewygodne i głośność jest irytująca.",
    "Zaakceptowałem ofertę i kupiłem nowy telefon, który działa bez zarzutu.",
    "Pisanie opinii o produkcie było dla mnie bardzo łatwe i szybkie. ",
    "One są wszystkie, luzacki, nudne, wporzadku, groźny, mieszane, fajny, zły, nie dobry,  dobra, pozytywne, piękne, smutne. ",
    "Całe to jebane zycie to jeden wielki dramat. ",
    "Chuj kurwa chuj. ",
]

for text in example_texts:
    predictions = nlp(text)
    inference_results.append({
        "text": text,
        "predictions": predictions
    })
    print(f"\nText: {text}")
    print("Inference Results:")
    print(predictions)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'PeftModelForTokenClassification' is not supported for token-classification. Supported models are ['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GemmaForTokenClassification', 'Gemma2ForTokenClassification'


Text: Nie jestem zadowolony z zakupu. Słuchawki są niewygodne i głośność jest irytująca.
Inference Results:
[{'entity': 'B-Positive', 'score': 0.19521159, 'index': 1, 'word': 'Nie</w>', 'start': 0, 'end': 3}, {'entity': 'B-Positive', 'score': 0.25935677, 'index': 3, 'word': 'zadowolony</w>', 'start': 11, 'end': 21}, {'entity': 'I-Positive', 'score': 0.1893232, 'index': 6, 'word': '.</w>', 'start': 30, 'end': 31}, {'entity': 'I-Neutral', 'score': 0.19215596, 'index': 10, 'word': 'są</w>', 'start': 42, 'end': 44}, {'entity': 'I-Neutral', 'score': 0.19994847, 'index': 12, 'word': 'godne</w>', 'start': 50, 'end': 55}, {'entity': 'I-Positive', 'score': 0.18524021, 'index': 13, 'word': 'i</w>', 'start': 56, 'end': 57}, {'entity': 'B-Neutral', 'score': 0.17517114, 'index': 17, 'word': 'i', 'start': 72, 'end': 73}, {'entity': 'I-Positive', 'score': 0.18837202, 'index': 20, 'word': '.</w>', 'start': 81, 'end': 82}]

Text: Zaakceptowałem ofertę i kupiłem nowy telefon, który działa bez zarzutu.


In [55]:
sentiment_colors = {
    'Negative': 'red',
    'Neutral': 'gray',
    'Positive': 'green'
}

In [56]:
def get_sentiment(label):
    """Extract the base sentiment from the label."""
    if label.startswith('B-') or label.startswith('I-'):
        return label.split('-', 1)[1]
    return label

for result in inference_results:
    text = result['text']
    predictions = result['predictions']
    
    words = text.split()
    
    sentiments = []
    scores = []
    
    word_sentiments = ['O'] * len(words)
    word_scores = [0.0] * len(words)
    
    for pred in predictions:
        label = pred['entity']
        sentiment = get_sentiment(label)
        score = pred['score']
        word = pred['word'].replace('</w>', '').strip()
        
        for idx, w in enumerate(words):
            clean_w = re.sub(r'[^\w]', '', w)
            if word.lower() == clean_w.lower():
                word_sentiments[idx] = sentiment
                word_scores[idx] = score
                break
    
    colors = [sentiment_colors.get(sentiment, 'black') for sentiment in word_sentiments]
    
    hover_texts = [f"Sentiment: {sentiment}<br>Score: {score:.2f}" 
                   for sentiment, score in zip(word_sentiments, word_scores)]
    
    fig = go.Figure()
    
    x = 0
    y = 0
    spacing = 0.5  # Adjust spacing between words
    
    for i, word in enumerate(words):
        fig.add_trace(go.Scatter(
            x=[x],
            y=[y],
            text=[word],
            mode='text',
            textfont=dict(color=colors[i], size=16),
            hoverinfo='text',
            hovertext=hover_texts[i],
            showlegend=False
        ))
        # Increment x position
        x += len(word) * 0.1 + spacing
    
    # Update layout
    fig.update_layout(
        title=f"Inference Results",
        xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
        yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
        margin=dict(l=20, r=20, t=50, b=20)
    )
    
    # Display the figure
    fig.show()